In [ ]:
# Import analysis packages:
import pandas as pd
import numpy as np

import random
from collections import Counter

# Import plotting packages:
from matplotlib import pyplot as plt
import seaborn as sns 

In [ ]:
# Import hypothesis-testing methods:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import chi2_contingency, f_oneway, kruskal
from scikit_posthocs import posthoc_dunn, posthoc_tukey_hsd
from pingouin import welch_anova, pairwise_gameshowell

# Import data pre-processing packages:
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler, LabelEncoder

# Import supervised ML classification classes:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Import performance-measuring methods:
from sklearn.metrics import classification_report, multilabel_confusion_matrix, confusion_matrix, plot_confusion_matrix

# documentation for multilabel_confusion_matrix:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.multilabel_confusion_matrix.html#sklearn.metrics.multilabel_confusion_matrix
# dis very important:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html

In [ ]:
''' I noticed that I was getting several warnings when compiling TensorFlow regarding appropriate
compiler flags; apparently, I'm not the only one, as evidenced by the following question from
stackoverflow:
https://stackoverflow.com/questions/66092421/how-to-rebuild-tensorflow-with-the-compiler-flags
I have chosen to implement the commenter's solution.'''
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import InputLayer, Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.metrics import Accuracy, Precision, Recall

In [ ]:
# The below is necessary for starting core Python generated random numbers
# in a well-defined state.
random.seed(11)

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.
np.random.seed(11)

# The below set_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/random/set_seed
tf.random.set_seed(11)

In [ ]:
forest_cover_data = pd.read_csv('cover_data.csv')

print(forest_cover_data.head(2))

In [ ]:
forest_cover_data = forest_cover_data.drop_duplicates()

In [ ]:
forest_cover_data["Cover_Type"] = forest_cover_data['class'].map({ 1:'Spruce/Fir',  2:'Lodgepole Pine',  \
                                                                   3:'Ponderosa Pine',  4:'Cottonwood/Willow',
                                                                   5:'Aspen', 6:'Douglas-fir', 7:'Krummholz'})

In [ ]:
print(forest_cover_data.info())

# Wow das a lotta columns

# Exploratory Data Analysis

In [ ]:
print(forest_cover_data['class'].value_counts())
print(forest_cover_data['Cover_Type'].value_counts())

In [ ]:
def get_summary_stats(column_name):
    
    # pluck off the data in column_name which belongs to each of the cover type classes, and store those
    # data in their own Series:
    temp_class1 = forest_cover_data[column_name][forest_cover_data['Cover_Type'] == 'Spruce/Fir']
    temp_class2 = forest_cover_data[column_name][forest_cover_data['Cover_Type'] == 'Lodgepole Pine']
    temp_class3 = forest_cover_data[column_name][forest_cover_data['Cover_Type'] == 'Ponderosa Pine']
    temp_class4 = forest_cover_data[column_name][forest_cover_data['Cover_Type'] == 'Cottonwood/Willow']
    temp_class5 = forest_cover_data[column_name][forest_cover_data['Cover_Type'] == 'Aspen']
    temp_class6 = forest_cover_data[column_name][forest_cover_data['Cover_Type'] == 'Douglas-fir']
    temp_class7 = forest_cover_data[column_name][forest_cover_data['Cover_Type'] == 'Krummholz']
    
    # get the data's quartiles:
    quarts_class1 = np.fix(np.quantile(temp_class1, [0.25,0.5,0.75]))
    quarts_class2 = np.fix(np.quantile(temp_class2, [0.25,0.5,0.75]))
    quarts_class3 = np.fix(np.quantile(temp_class3, [0.25,0.5,0.75]))
    quarts_class4 = np.fix(np.quantile(temp_class4, [0.25,0.5,0.75]))
    quarts_class5 = np.fix(np.quantile(temp_class5, [0.25,0.5,0.75]))
    quarts_class6 = np.fix(np.quantile(temp_class6, [0.25,0.5,0.75]))
    quarts_class7 = np.fix(np.quantile(temp_class7, [0.25,0.5,0.75]))
    
    # gather up a dictionary for later conversion to a DataFrame:
    tempdict = {'Cover_Type':['Spruce/Fir', 'Lodgepole Pine','Ponderosa Pine','Cottonwood/Willow','Aspen',\
                              'Douglas-fir','Krummholz'],\
                # calculate the average value of column_name for each class, and round to the nearest whole number:
                'Avg':['{:.0f}'.format(temp_class1.mean()),'{:.0f}'.format(temp_class2.mean()),\
                       '{:.0f}'.format(temp_class3.mean()),'{:.0f}'.format(temp_class4.mean()),\
                       '{:.0f}'.format(temp_class5.mean()),'{:.0f}'.format(temp_class6.mean()),\
                       '{:.0f}'.format(temp_class7.mean())],\
                # calculate the median value of column_name for each class, and round to the nearest whole number:
                'Median':['{:.0f}'.format(temp_class1.median()),'{:.0f}'.format(temp_class2.median()),\
                          '{:.0f}'.format(temp_class3.median()),'{:.0f}'.format(temp_class4.median()),\
                          '{:.0f}'.format(temp_class5.median()),'{:.0f}'.format(temp_class6.median()),\
                          '{:.0f}'.format(temp_class7.median())],\
                # find the minimum value of column_name for each class:
                'Min':[min(temp_class1),min(temp_class2),min(temp_class3),min(temp_class4),\
                              min(temp_class5),min(temp_class6),min(temp_class7)],\
                # find the maximum value of column_name for each class:
                'Max':[max(temp_class1),max(temp_class2),max(temp_class3),max(temp_class4),\
                              max(temp_class5),max(temp_class6),max(temp_class7)],\
                # gather up the quartiles calculated above:
                'Quartiles':[quarts_class1,quarts_class2,quarts_class3,quarts_class4,\
                             quarts_class5,quarts_class6,quarts_class7],\
                # calculate the interquartile range (IQR) for each class:
               'IQR':[quarts_class1[2]-quarts_class1[0],quarts_class2[2]-quarts_class2[0],\
                      quarts_class3[2]-quarts_class3[0],quarts_class4[2]-quarts_class4[0],\
                      quarts_class5[2]-quarts_class5[0],quarts_class6[2]-quarts_class6[0],\
                      quarts_class7[2]-quarts_class7[0]]}
    
    # convert from a dictionary to a DataFrame:
    df_to_return = pd.DataFrame(tempdict)

    # return the DataFrame:
    return df_to_return

# dis idea from
# https://kiwidamien.github.io/stylish-pandas.html

In [ ]:
'''
Dis code scrap storage:

                ' 25th Per.':['{:.0f}'.format(np.percentile(temp_class1,0.25)),\
                             '{:.0f}'.format(np.percentile(temp_class2,0.25)),\
                             '{:.0f}'.format(np.percentile(temp_class3,0.25)),\
                             '{:.0f}'.format(np.percentile(temp_class4,0.25)),\
                             '{:.0f}'.format(np.percentile(temp_class5,0.25)),\
                             '{:.0f}'.format(np.percentile(temp_class6,0.25)),\
                             '{:.0f}'.format(np.percentile(temp_class7,0.25))],\
                ' 75th Per.':['{:.0f}'.format(np.percentile(temp_class1,0.75)),\
                             '{:.0f}'.format(np.percentile(temp_class2,0.75)),\
                             '{:.0f}'.format(np.percentile(temp_class3,0.75)),\
                             '{:.0f}'.format(np.percentile(temp_class4,0.75)),\
                             '{:.0f}'.format(np.percentile(temp_class5,0.75)),\
                             '{:.0f}'.format(np.percentile(temp_class6,0.75)),\
                             '{:.0f}'.format(np.percentile(temp_class7,0.75))]}

'''

In [ ]:
def plot_a_distribution(column_name, nbins = 40, numofbins=40):
    sns.set_context("notebook", font_scale=0.8, rc={"lines.linewidth": 1.5})
    
    plt.figure(figsize=(9,5))
    if column_name == 'Slope':
        sns.histplot(forest_cover_data[column_name], bins = nbins, kde = True, discrete = True)
    else:
        sns.histplot(forest_cover_data[column_name], bins = nbins, kde = True)
    if (column_name == 'Aspect') or (column_name == 'Slope'):
        plt.xlabel(column_name + ' [degrees]', fontsize=13)
    elif (column_name == 'Hillshade_9am') or (column_name == 'Hillshade_Noon') or (column_name == 'Hillshade_3pm'):
        plt.xlabel(column_name, fontsize=13)
    else:
        plt.xlabel(column_name + ' [m]', fontsize=13)
    plt.ylabel('Counts per Bin', fontsize=13)
    plt.title(column_name + ' Distribution', fontsize=18)
    
    plt.figure(figsize=(16,10))
    sns.violinplot(data = forest_cover_data, x = 'Cover_Type', y = column_name, \
                   order = ['Spruce/Fir', 'Lodgepole Pine', 'Ponderosa Pine', 'Cottonwood/Willow', \
                            'Aspen', 'Douglas-fir','Krummholz'])
    plt.xlabel('Class', fontsize=13)
    if (column_name == 'Aspect') or (column_name == 'Hillshade_9am') or \
    (column_name == 'Hillshade_noon') or (column_name == 'Hillshade_3pm'):
        plt.ylabel(column_name, fontsize=13)
    else:
        plt.ylabel(column_name + ' [m]', fontsize=13)    
    plt.title('Violin Plot of the ' + column_name + ' Distribution, Split by Class', fontsize=18)

    temp_class1 = forest_cover_data[column_name][forest_cover_data['class'] == 1]
    temp_class2 = forest_cover_data[column_name][forest_cover_data['class'] == 2]
    temp_class3 = forest_cover_data[column_name][forest_cover_data['class'] == 3]
    temp_class4 = forest_cover_data[column_name][forest_cover_data['class'] == 4]
    temp_class5 = forest_cover_data[column_name][forest_cover_data['class'] == 5]
    temp_class6 = forest_cover_data[column_name][forest_cover_data['class'] == 6]
    temp_class7 = forest_cover_data[column_name][forest_cover_data['class'] == 7]
   
    plt.figure(figsize=(16,5))
    if column_name == 'Slope':
        sns.histplot(temp_class1, bins = numofbins, stat = 'density', kde = True, alpha = 0.50, color = 'rosybrown', \
                     discrete = True, common_norm = False)
        sns.histplot(temp_class2, bins = numofbins, stat = 'density', kde = True, alpha = 0.50, color = 'olivedrab', \
                     discrete = True, common_norm = False)
        sns.histplot(temp_class3, bins = numofbins, stat = 'density', kde = True, alpha = 0.50, color = 'teal', \
                     discrete = True, common_norm = False)
        sns.histplot(temp_class4, bins = numofbins, stat = 'density', kde = True, alpha = 0.50, color = 'silver', \
                     discrete = True, common_norm = False)
        sns.histplot(temp_class5, bins = numofbins, stat = 'density', kde = True, alpha = 0.50, color = 'purple', \
                     discrete = True, common_norm = False)
        sns.histplot(temp_class6, bins = numofbins, stat = 'density', kde = True, alpha = 0.50, color = 'maroon', \
                     discrete = True, common_norm = False)
        sns.histplot(temp_class7, bins = numofbins, stat = 'density', kde = True, alpha = 0.50, color = 'darkblue', \
                     discrete = True, common_norm = False)
    else:
        sns.histplot(temp_class1, bins = numofbins, stat = 'density', kde = True, alpha = 0.50, color = 'rosybrown', \
                 common_norm = False)
        sns.histplot(temp_class2, bins = numofbins, stat = 'density', kde = True, alpha = 0.50, color = 'olivedrab', \
                 common_norm = False)
        sns.histplot(temp_class3, bins = numofbins, stat = 'density', kde = True, alpha = 0.50, color = 'teal', \
                 common_norm = False)
        if column_name == 'Horizontal_Distance_To_Hydrology':
            sns.histplot(temp_class4, bins = 20, stat = 'density', kde = True, alpha = 0.50, color = 'silver', \
                         common_norm = False)
        elif column_name == 'Vertical_Distance_To_Hydrology':
            sns.histplot(temp_class4, bins = 20, stat = 'density', kde = True, alpha = 0.50, color = 'silver', \
                         common_norm = False)
        else:
            sns.histplot(temp_class4, bins = numofbins, stat = 'density', kde = True, alpha = 0.50, color = 'silver', \
                         common_norm = False)
        sns.histplot(temp_class5, bins = numofbins, stat = 'density', kde = True, alpha = 0.50, color = 'purple', \
                     common_norm = False)
        sns.histplot(temp_class6, bins = numofbins, stat = 'density', kde = True, alpha = 0.50, color = 'maroon', \
                     common_norm = False)
        sns.histplot(temp_class7, bins = numofbins, stat = 'density', kde = True, alpha = 0.50, color = 'darkblue', \
                     common_norm = False)
    plt.legend(['class 1', 'class 2', 'class 3', 'class 4', 'class 5', 'class 6', 'class 7'], fontsize=13)
    if (column_name == 'Aspect') or (column_name == 'Hillshade_9am') or \
    (column_name == 'Hillshade_noon') or (column_name == 'Hillshade_3pm'):
        plt.xlabel(column_name, fontsize=13)
    else:
        plt.xlabel(column_name + ' [m]', fontsize=13)
    plt.ylabel('Density', fontsize=13)
    plt.title(column_name + ' Distribution, Split by Class', fontsize=18)

In [ ]:
# dis idea from
# https://kiwidamien.github.io/stylish-pandas.html

def format_float(value):
    return f'{value:,.2E}'

pd.options.display.float_format = format_float

In [ ]:
def test_for_association_kwh(column_name):
    
    temp_class1 = forest_cover_data[column_name][forest_cover_data['class'] == 1]
    temp_class2 = forest_cover_data[column_name][forest_cover_data['class'] == 2]
    temp_class3 = forest_cover_data[column_name][forest_cover_data['class'] == 3]
    temp_class4 = forest_cover_data[column_name][forest_cover_data['class'] == 4]
    temp_class5 = forest_cover_data[column_name][forest_cover_data['class'] == 5]
    temp_class6 = forest_cover_data[column_name][forest_cover_data['class'] == 6]
    temp_class7 = forest_cover_data[column_name][forest_cover_data['class'] == 7]
    
    kh_tstat, kh_pval = kruskal(temp_class1, temp_class2, temp_class3, temp_class4, \
                                temp_class5, temp_class6, temp_class7)
    print('Kruskal-Wallis H-test p-value for ' + column_name + '/class association: '+ str(kh_pval))
    
    dunntestresult = posthoc_dunn(a=forest_cover_data, val_col = column_name, group_col = 'Cover_Type', sort = True)
    print('')
    print("Dunn's pairwise test for multiple comparisons of mean rank sums:")
    plt.figure(figsize = (10,7))
    sns.heatmap(dunntestresult, annot=True, center=0.001, fmt=".2E")

# dis very helpful:
#https://kiwidamien.github.io/stylish-pandas.html
# also dis:
#https://stackoverflow.com/questions/6913532/display-a-decimal-in-scientific-notation

In [ ]:
# for elevation, cannot use K-H test: different variances, different shapes of distributions, according to dis website:
# http://www.biostathandbook.com/kruskalwallis.html
# later on he recommends a Welch's ANOVA test:
# http://www.biostathandbook.com/onewayanova.html#welch
# From here:
# https://stackoverflow.com/questions/50964427/welchs-anova-in-python
# I was lead to 
# https://pingouin-stats.org/index.html
# with documentation at:
# https://pingouin-stats.org/generated/pingouin.welch_anova.html

def test_for_association_welch(column_name):
    
    anal_of_var = welch_anova(data=forest_cover_data, dv=column_name, between='Cover_Type')
    
    pval_df = anal_of_var['p-unc']
    pval = pval_df.iloc[0]
    
    print('Welch ANOVA p-value for ' + column_name + '/class association: '+ str(pval))
    
    gameshowelltestresult = pairwise_gameshowell(data=forest_cover_data, dv=column_name, between='Cover_Type').round(3)
    ghtestresult = gameshowelltestresult[['A','B','mean(A)','mean(B)','T','pval']]
    print('')
    print("Games-Howell pairwise test for multiple comparisons of means:")
    print(ghtestresult)

## Elevation

In [ ]:
elev_stats = get_summary_stats('Elevation')
print(elev_stats)

In [ ]:
plot_a_distribution('Elevation', nbins=100)

In [ ]:
elev_pval, elev_dunntestresult = test_for_association('Elevation')
print('Kruskal-Wallis H-test p-value for Elevation/class association: '+ str(elev_pval))
print("Dunn's pairwise test for multiple comparisons of mean rank sums:")
print(elev_dunntestresult)
# dis very helpful:
#https://stackoverflow.com/questions/4288973/whats-the-difference-between-s-and-d-in-python-string-formatting/56382046

## Aspect

In [ ]:
# dis what aspect is:
# https://en.wikipedia.org/wiki/Aspect_(geography)
plot_a_distribution('Aspect')

In [ ]:
aspect_pval, aspect_dunntestresult = test_for_association('Aspect')
print('Kruskal-Wallis H-test p-value for Aspect/class association: '+ str(aspect_pval))
print("Dunn's pairwise test for multiple comparisons of mean rank sums:")
print(aspect_dunntestresult)

## Slope

In [ ]:
plot_a_distribution('Slope', nbins=70, numofbins=70)

In [ ]:
slope_pval, slope_dunntestresult = test_for_association('Slope')
print('Kruskal-Wallis H-test p-value for Slope/class association: '+ str(slope_pval))
print("Dunn's pairwise test for multiple comparisons of mean rank sums:")
print(slope_dunntestresult)

## Horizontal Distance to Hydrology

In [ ]:
plot_a_distribution('Horizontal_Distance_To_Hydrology', nbins=30, numofbins=20)

In [ ]:
hd2h_pval, hd2h_dunntestresult = test_for_association('Horizontal_Distance_To_Hydrology')
print('Kruskal-Wallis H-test p-value for H.D.T.H./class association: '+ str(hd2h_pval))
print("Dunn's pairwise test for multiple comparisons of mean rank sums:")
print(hd2h_dunntestresult)

## Vertical Distance to Hydrology

In [ ]:
plot_a_distribution('Vertical_Distance_To_Hydrology', numofbins=30)

In [ ]:
vd2h_pval, vd2h_dunntestresult = test_for_association('Vertical_Distance_To_Hydrology')
print('Kruskal-Wallis H-test p-value for V.D.T.H./class association: '+ str(vd2h_pval))
print("Dunn's pairwise test for multiple comparisons of mean rank sums:")
print(vd2h_dunntestresult)

## Horizontal Distance to Roadways

In [ ]:
plot_a_distribution('Horizontal_Distance_To_Roadways')

In [ ]:
hd2r_pval, hd2r_dunntestresult = test_for_association('Horizontal_Distance_To_Roadways')
print('Kruskal-Wallis H-test p-value for H.D.T.R./class association: '+ str(hd2r_pval))
print("Dunn's pairwise test for multiple comparisons of mean rank sums:")
print(hd2r_dunntestresult)

## Hill Shade at 9AM

In [ ]:
plot_a_distribution('Hillshade_9am')

In [ ]:
shade0900_pval, shade0900_dunntestresult = test_for_association('Hillshade_9am')
print('Kruskal-Wallis H-test p-value for Hill Shade at 9AM/class association: '+ str(shade0900_pval))
print("Dunn's pairwise test for multiple comparisons of mean rank sums:")
print(shade0900_dunntestresult)

## Hill Shade at 12PM

In [ ]:
plot_a_distribution('Hillshade_Noon')

In [ ]:
shade1200_pval, shade1200_dunntestresult = test_for_association('Hillshade_Noon')
print('Kruskal-Wallis H-test p-value for Hill Shade at 12PM/class association: '+ str(shade1200_pval))
print("Dunn's pairwise test for multiple comparisons of mean rank sums:")
print(shade1200_dunntestresult)

## Hill Shade at 3PM

In [ ]:
plot_a_distribution('Hillshade_3pm')

In [ ]:
shade1500_pval, shade1500_dunntestresult = test_for_association('Hillshade_3pm')
print('Kruskal-Wallis H-test p-value for Hill Shade at 3PM/class association: '+ str(shade1500_pval))
print("Dunn's pairwise test for multiple comparisons of mean rank sums:")
print(shade1500_dunntestresult)

## Horizontal Distance to Fire Points

In [ ]:
plot_a_distribution('Horizontal_Distance_To_Fire_Points')

In [ ]:
hdtfp_pval, hdtfp_dunntestresult = test_for_association('Horizontal_Distance_To_Fire_Points')
print('Kruskal-Wallis H-test p-value for H.D.T.F.P./class association: '+ str(hdtfp_pval))
print("Dunn's pairwise test for multiple comparisons of mean rank sums:")
print(hdtfp_dunntestresult)

## Wilderness Area 

In [ ]:
print(forest_cover_data.Wilderness_Area1.value_counts())

## Soil Type

In [ ]:
print(forest_cover_data.Soil_Type1.value_counts())

# Data PreProcessing

In [ ]:
variables = forest_cover_data.iloc[:,0:53]
labels = forest_cover_data.iloc[:,-1]

labels_list = labels.to_list()

ratio_train = 0.7
ratio_valid = 0.15
ratio_test = 0.15
ratio_valid_adjusted = ratio_valid / (1 - ratio_test)

vars_train, vars_test, labels_train, labels_test = \
    train_test_split(variables, labels, test_size = ratio_test, stratify = labels_list)

print(labels.value_counts()/len(labels))
print(labels_train.value_counts()/len(labels_train))
print(labels_test.value_counts()/len(labels_test))

In [ ]:
coltransform = ColumnTransformer([('minmax', MinMaxScaler(), \
                                 ['Elevation', 'Aspect', 'Slope', \
                                  'Horizontal_Distance_To_Hydrology', \
                                  'Vertical_Distance_To_Hydrology', \
                                  'Horizontal_Distance_To_Roadways', \
                                  'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', \
                                  'Horizontal_Distance_To_Fire_Points'])], \
                                  remainder='passthrough')
vars_scaled_train = coltransform.fit_transform(vars_train)
vars_scaled_test = coltransform.transform(vars_test)

In [ ]:
le=LabelEncoder()
labelz_train=le.fit_transform(labels_train.astype(str))
labelz_test=le.transform(labels_test.astype(str))

labelz_train = to_categorical(labelz_train, dtype='int64')
labelz_test = to_categorical(labelz_test, dtype='int64')

In [ ]:
print(vars_scaled_train.shape, labelz_train.shape)
print('')
print(len(vars_scaled_train)*ratio_valid_adjusted)
print('')
print(vars_scaled_test.shape, labelz_test.shape)

# Building a model

In [ ]:
def build_a_model(xs):
    
    nvars = xs.shape[1]

    thismodel = Sequential()
    thismodel.add(InputLayer(input_shape=(nvars,)))
    thismodel.add(Dense(512, activation='relu'))  # 512
    thismodel.add(Dropout(0.01))
    thismodel.add(Dense(256, activation='relu'))  #256
    thismodel.add(Dropout(0.01))
    thismodel.add(Dense(128, activation='relu'))  #128
    thismodel.add(Dropout(0.01))
    thismodel.add(Dense(64, activation='relu'))  #64
    thismodel.add(Dropout(0.01))
    thismodel.add(Dense(32, activation='relu')) #32
    thismodel.add(Dropout(0.001))
    thismodel.add(Dense(16, activation='relu')) #16
    thismodel.add(Dropout(0.01))
    thismodel.add(Dense(7, activation='softmax'))
    thisoptimizer = Adam(learning_rate=0.005)  #0.005
    thismodel.compile(loss='CategoricalCrossentropy',  metrics=['accuracy','Precision','Recall'], \
                      optimizer=thisoptimizer)
    
    return thismodel

In [ ]:
earlistop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=40)

In [ ]:
nn_model = build_a_model(vars_scaled_train)

In [ ]:
model_history = nn_model.fit(vars_scaled_train, labelz_train, \
                             validation_split = ratio_valid_adjusted, \
                             shuffle = False, epochs = 310, batch_size = 800, \
                             callbacks=[earlistop], verbose = 1)
model_predictions = nn_model.predict(vars_scaled_test)

model_crossentropy, model_accuracy, model_precision, model_recall = \
     nn_model.evaluate(vars_scaled_test, labelz_test, verbose = 0)

In [ ]:
plt.figure(figsize=(32,16))
plt.subplots_adjust(hspace=0.2,wspace=0.2)

plt.subplot(2, 2, 1)
plt.plot(model_history.history['loss'], color = 'blue')
plt.plot(model_history.history['val_loss'], color = 'crimson')
plt.title('Model Categorical Cross-Entropy')
plt.ylabel('Categorical Cross-Entropy')
plt.xlabel('Epoch')
plt.legend(['Training Data', 'Validation Data'], loc='upper right')

plt.subplot(2, 2, 2)
plt.plot(model_history.history['accuracy'], color = 'blue')
plt.plot(model_history.history['val_accuracy'], color = 'crimson')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training Data', 'Validation Data'], loc='lower right')

plt.subplot(2, 2, 3)
plt.plot(model_history.history['precision'], color = 'blue')
plt.plot(model_history.history['val_precision'], color = 'crimson')
plt.title('Model Precision')
plt.ylabel('Precision')
plt.xlabel('Epoch')
plt.legend(['Training Data', 'Validation Data'], loc='lower right')

plt.subplot(2, 2, 4)
plt.plot(model_history.history['recall'], color = 'blue')
plt.plot(model_history.history['val_recall'], color = 'crimson')
plt.title('Model Recall')
plt.ylabel('Recall')
plt.xlabel('Epoch')
plt.legend(['Training Data', 'Validation Data'], loc='lower right')

In [ ]:
# https://www.tensorflow.org/guide/keras/save_and_serialize
training_model.save('cover_model.h5')

In [ ]:
pred_cover_classes = np.argmax(model_predictions, axis = 1)
true_cover_classes = np.argmax(labelz_test, axis=1)
print(classification_report(true_cover_classes, pred_cover_classes, zero_division = 'warn'))

# in multiclass tasks, labels are binarized under a one-vs-rest way
# In multilabel confusion matrix MCM ...
# the count of true negatives is MCM[0,0]
# false negatives is MCM [1,0]
# true positives is MCM [1,1]
# and false positives is MCM [0,1].
# nomenclature: [row,column]
#print(multilabel_confusion_matrix(true_cover_classes, pred_cover_classes))
# The above was a decent idea but not apporpirate for this project.

In [ ]:
confuse_mat = confusion_matrix(true_cover_classes, pred_cover_classes)
print(confuse_mat)
# from https://stackoverflow.com/questions/35572000/how-can-i-plot-a-confusion-matrix
# https://seaborn.pydata.org/generated/seaborn.heatmap.html

cover_types = ['Spruce/Fir', 'Lodgepole Pine', 'Ponderosa Pine', 'Cottonwood/Willow', 'Aspen', 'Douglas-fir',\
               'Krummholz']

confuse_df = pd.DataFrame(confuse_mat, index = cover_types, columns = cover_types)
plt.figure(figsize = (10,7))
sns.heatmap(confuse_df, annot=True, center=1000, fmt="d")
plt.show()